In [1]:
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np
import cv2
import mediapipe as mp
import torch
import pandas as pd
from scipy.fft import fft, ifft


In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")
torch.cuda.empty_cache()

Using cuda device


In [3]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

def draw_landmarks_on_image(rgb_image, detection_result):
  pose_landmarks_list = detection_result.pose_landmarks
  annotated_image = np.copy(rgb_image)

  # Loop through the detected poses to visualize.
  for idx in range(len(pose_landmarks_list)):
    pose_landmarks = pose_landmarks_list[idx]

    # Draw the pose landmarks.
    pose_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    pose_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in pose_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      pose_landmarks_proto,
      solutions.pose.POSE_CONNECTIONS,
      solutions.drawing_styles.get_default_pose_landmarks_style())
  return annotated_image



def getFiturLandmark(img):
    err = None
    detection_result = []
    try:
        
        base_options = python.BaseOptions(model_asset_path='pose_landmarker_heavy.task')
        options = vision.PoseLandmarkerOptions(
        base_options=base_options,
        output_segmentation_masks=True)
        detector = vision.PoseLandmarker.create_from_options(options)
        
        detection_result = detector.detect(img)

        err = None
    
    except:
        err = 1
        detection_result = []
    
    return detection_result, err



    
    # concate 
def flatten(data):
    a = np.ravel(data)
    return a


In [4]:
from torchvision import transforms
train_transform = transforms.Compose([
    transforms.ToPILImage(),
     transforms.RandomRotation(degrees=(30, 70)),
    transforms.Resize((100, 100)),
    # transforms.Grayscale(1),
    # transforms.RandomHorizontalFlip(p=0.5),
    # transforms.RandomVerticalFlip(p=0.5),
    # transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
   
    transforms.ToTensor(),
   
])

In [5]:
# create dataset class
import os
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from sklearn.cluster import KMeans
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence


def toTensor(_x):
    # _em = torch.rand(168,51)
    # _newX = torch.as_tensor(_x)
    # _padSeq = pad_sequence([_em, _newX], batch_first=True, padding_value=0)
    # _xDt = pd.Series(_x)
    return _x



class dataClass(Dataset):
    def __init__(self, pickle_data, root_dir, transform=None):
        self.file = pd.read_pickle(pickle_data)
        self.input = torch.tensor
        self.root_dir = root_dir
        self.transform = transform
        
        self.yoloTensor = [torch.tensor(_x) for _x in self.file['YOLO11']]
        self.yoloTensorPad = pad_sequence(self.yoloTensor, batch_first=True)
        self.leng = [len(_x) for _x in self.yoloTensorPad]
        self.yoloTensorPackPad = pack_padded_sequence(self.yoloTensorPad, lengths=self.leng, batch_first=True, enforce_sorted=False)
        
    
    def __len__(self):
        return len(self.file)
    
    def __getitem__(self, index):
        _fitur = self.file.iloc[index]['YOLO11']
        _file = self.file.iloc[index]['file']
        _image = self.file.iloc[index]['image']
        
        _class = self.file.iloc[index]['class']
        # _dfposeFlatternFFT = np.fft.fft(_dfposeFlatten)
        _datareturn = {
                    'dataInput': _fitur,
                    'target': _class,
                    'file': _file,
                    'image': _image
                       }
        
            
        return _datareturn

data_torch = dataClass(pickle_data="../../dataset/pickle_YOLO", root_dir="../../dataset/extractFrame", transform=None)


In [12]:
aaa = pd.read_pickle("../../dataset/pickle_YOLO")
# aaa

In [ ]:
oo = [torch.tensor(x) for x in aaa['YOLO11']]


C:\Users\intel\AppData\Local\Temp\ipykernel_11740\2821895653.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:281.)
  oo = [torch.tensor(x) for x in aaa['YOLO11']]


[tensor([[8.8629e+01, 1.9603e+01, 9.9672e-01,  ..., 0.0000e+00, 0.0000e+00,
          5.4132e-03],
         [8.8524e+01, 1.9664e+01, 9.9683e-01,  ..., 0.0000e+00, 0.0000e+00,
          4.9574e-03],
         [8.8532e+01, 1.9642e+01, 9.9695e-01,  ..., 0.0000e+00, 0.0000e+00,
          7.1495e-03],
         ...,
         [8.9003e+01, 1.7431e+01, 9.9672e-01,  ..., 0.0000e+00, 0.0000e+00,
          3.6417e-03],
         [8.8981e+01, 1.7465e+01, 9.9678e-01,  ..., 0.0000e+00, 0.0000e+00,
          2.9118e-03],
         [8.8902e+01, 1.7464e+01, 9.9646e-01,  ..., 0.0000e+00, 0.0000e+00,
          3.4202e-03]]),
 tensor([[6.8399e+01, 3.2470e+01, 9.9599e-01,  ..., 0.0000e+00, 0.0000e+00,
          2.3484e-03],
         [6.8343e+01, 3.2494e+01, 9.9613e-01,  ..., 0.0000e+00, 0.0000e+00,
          2.0204e-03],
         [6.8435e+01, 3.2619e+01, 9.9587e-01,  ..., 0.0000e+00, 0.0000e+00,
          2.3180e-03],
         ...,
         [6.9627e+01, 3.2303e+01, 9.9698e-01,  ..., 0.0000e+00, 0.0000e+00,
   

In [24]:
print(oo[0].shape)
print(oo[1].shape)
ooPad = pad_sequence(oo, batch_first=True)
leng = [len(_u) for _u in ooPad]
oopadPack = pack_padded_sequence(ooPad, lengths=leng, batch_first=True, enforce_sorted=False)
print(oopadPack[0].shape)
print(oopadPack[1].shape)



torch.Size([50, 51])
torch.Size([120, 51])
torch.Size([92568, 51])
torch.Size([168])


In [8]:
from torch.utils.data import DataLoader, random_split

# split dataset
print(len(data_torch))
train_data, val_data = random_split(data_torch, [385, 166])
print(type(train_data[0]['dataInput']))

train_dataloader = DataLoader(train_data, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=64, shuffle=True)
print(train_dataloader)


551
<class 'list'>


In [9]:
# model
import torch #pytorch
import torch.nn as nn
from torch.autograd import Variable 

class LSTM1(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):
        super(LSTM1, self).__init__()
        self.num_classes = num_classes #number of classes
        self.num_layers = num_layers #number of layers
        self.input_size = input_size #input size
        self.hidden_size = hidden_size #hidden state
        self.seq_length = seq_length #sequence length

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True) #lstm
        self.fc_1 =  nn.Linear(hidden_size, 128) #fully connected 1
        self.fc = nn.Linear(128, num_classes) #fully connected last layer

        self.relu = nn.ReLU()
    
    def forward(self,x):
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) #hidden state
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) #internal state
        # Propagate input through LSTM
        output, (hn, cn) = self.lstm(x, (h_0, c_0)) #lstm with input, hidden, and internal state
        hn = hn.view(-1, self.hidden_size) #reshaping the data for Dense layer next
        out = self.relu(hn)
        out = self.fc_1(out) #first Dense
        out = self.relu(out) #relu
        out = self.fc(out) #Final Output
        return out

In [10]:
# train
num_epochs = 1000 #1000 epochs
learning_rate = 0.001 #0.001 lr

input_size = 4 #number of features
hidden_size = 2 #number of features in hidden state
num_layers = 1 #number of stacked lstm layers

num_classes = 1 #number of output classes 


lstm1 = LSTM1(num_classes, input_size, hidden_size, num_layers, X_train_tensors_final.shape[1]) #our lstm class sss


NameError: name 'X_train_tensors_final' is not defined